In [ ]:
!pip install visualkeras

In [ ]:
print(tf.__version__)

In [ ]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import multilabel_confusion_matrix, classification_report

import zipfile

from sklearn.utils import class_weight

In [ ]:
with zipfile.ZipFile("images_laplacian.zip","r") as zip_ref:
    zip_ref.extractall("")

## Load data

In [ ]:
batch_size = 32
img_height = 299
img_width = 299

## Create base Xception model

In [ ]:
def vgg_block(layer_in, n_filters, n_conv):
    # add convolutional layers
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
    # add max pooling layer
    layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
    return layer_in

In [ ]:
# create the base pre-trained model
base_model = ResNet152V2(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = vgg_block(x, 64, 2)
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
train_ds_pn = tf.keras.utils.image_dataset_from_directory(
  "images_laplacian",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=None)

In [ ]:
val_ds_pn = tf.keras.utils.image_dataset_from_directory(
  "images_laplacian",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=None)

In [ ]:
train_x = []
train_y = []

for image, label in train_ds_pn:
    train_x.append(image)
    train_y.append(label)

train_x = np.array(train_x)
train_y = np.array(train_y)

In [ ]:
val_x = []
val_y = []

for image, label in val_ds_pn:
    val_x.append(image)
    val_y.append(label)

val_x = np.array(val_x)
val_y = np.array(val_y)

In [ ]:
train_x = tf.keras.applications.resnet_v2.preprocess_input(train_x)
val_x = tf.keras.applications.resnet_v2.preprocess_input(val_x)

In [ ]:
class_weights = class_weight.compute_class_weight(
           class_weight='balanced',
           classes=np.unique(train_y), 
           y=train_y)

class_weights = {i:class_weights[i] for i in range(3)}

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-05),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
datagen = ImageDataGenerator(width_shift_range=0.2, 
                             height_shift_range=0.2, 
                             horizontal_flip=True, 
                             shear_range=0.2, 
                             rotation_range=20, 
                             zoom_range=0.2, 
                             fill_mode='nearest')

# prepare iterator
it_train = datagen.flow(train_x, train_y, batch_size=16)

# fit model
steps = int(train_x.shape[0] / 16)
hist = model.fit_generator(it_train, steps_per_epoch=steps, epochs=20, validation_data=(val_x, val_y), verbose=1, class_weight=class_weights)

# evaluate model
_, acc = model.evaluate(val_x, val_y, verbose=0)
print('> %.3f' % (acc * 100.0))

In [ ]:
# model.save('model_xception_scratch')

In [ ]:
# import shutil
# shutil.make_archive('model_xception_scratch', 'zip', 'model_xception_scratch')

In [ ]:
plt.style.use("classic")
plt.figure(figsize=(16, 9))
plt.plot(hist.history['loss'], label="Train Loss")
plt.plot(hist.history['val_loss'], label="Valid Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss over the Epochs (ResNet 152V2)")
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(16, 9))
plt.plot(hist.history['accuracy'], label="Train Accuracy")
plt.plot(hist.history['val_accuracy'], label="Valid Accuracy")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy over the Epochs (ResNet 152V2)")
plt.show()

## Evaluate

In [ ]:
y_pred = model.predict(val_x)
y_pred_max = np.argmax(y_pred, axis=1)
print(classification_report(val_y, y_pred_max))

In [ ]:
print(multilabel_confusion_matrix(val_y, y_pred_max))

In [ ]:
import pandas as pd
pd.Series(val_y).value_counts()

In [ ]:
# weighted avg specificity
(544 * (585/(585+42)) + 325 * (832/(832+14)) + 302 * (762/(762+107))) / (544+325+302)

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.figure(figsize=(12,8))
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='validation')
    plt.legend()
    
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='validation')
    plt.legend()

In [ ]:
summarize_diagnostics(history)